In [1]:
import numpy as np
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import chain
import random
import math

In [2]:
# amp
def ampFn(amp_type,t,t0,T,amp,aparam):
    # t0: initial time
    # T: length of time for sound
    # t1: time of peak
    # a1: trancation exponent(a1=1: NoEffect, 1<<a1: too tracate)
    # a2: trancation exponent(0<a2, 0<a2<<1: too tracate, 1<<a2: NoEffect
    # a1=-1&a2=1: convex trancation shape
    t1 = aparam[0]
    a1 = aparam[1]
    a2 = aparam[2]
    if amp_type == 1:
        if t <= t0:
            ampVal = 0
        else:
            if t <= t0 + t1:
                ampVal = amp*(t-t0)/t1
            else:
                ampVal = amp*(-(t-T)/T)*((t1+a2)/(t-t0+a2))**a1

    return(ampVal)


def makeampFnwave(amp, freq,T, dT, sr, aparam):
    outwave = []
#    dT = 0.2
    T = np.around(T/dT)*dT
    dt = 1/sr
    t0 = 0
    for t in np.arange( 0., T+dt, dt ) :
        w = ampFn(1, t, t0, T, amp, aparam)*np.sin(2*np.pi*freq*t)
        outwave.append(w)
    return(outwave)

In [3]:
import wave
import struct

def makewave(amp,freq,sec,sr):
    outwave = []
    for i in range( int(sec*sr) ) :
        w = amp*np.sin(2*np.pi*freq*i/sr)
        outwave.append(w)
    return(outwave)


def wavwrite(inputlist,filename):
    maxamp = 32767.0
    int16wave = [int(x * maxamp) for x in inputlist]
    binwave = struct.pack("h"*len(int16wave),*int16wave)

    nchannnles = 1 # 1=monoral , 2=stereo
    sampwitdth = 2 # 1=8bit,2=16bit,3=,...
    framerate = 44100 # sampling rate ex.44100Hz
    nframes = len(inputlist) # framerate * sec
    
    of = wave.open(filename,"w")
    of.setparams((nchannnles,sampwitdth,framerate,nframes,"NONE","not compressed"))
    of.writeframes(binwave)
    of.close

In [4]:
Tbase = 0.5
dT =0.2
sr = 44100 # sampling rate ex.44100Hz

wave_temp = []
# d l m f s r c 
d, dsh, l, lsh, m, f, fsh, s, ssh, r, rsh, c, d2, d2sh, l2 = range(39,54)
keys =     [   r,  fsh,    m,  l,  fsh,    s,     r,      c, r,   c,     s, fsh,    m, s,    r,      c,c,d2sh,  c,d2sh,     c,     r,      s,    s,  fsh,   m, l] # good 
#keys =     [ssh,     f,  lsh,  l,  fsh,    s,   rsh,      c, r,   c,     s,    f,  lsh, s, rsh,     c,c,        l,  c,d2sh,     c,   ssh,      s,    s,   f,   m, l] 
Tratios = [   1,1/2,1/2, 1,1/2,1/2,3/2,1/2,2,   1,1/2,1/2,1/2,1,1/2,1/2,1,  1/2, 2,       1,1/2,1/2,1/2,1/2,1/2,1/2,4]
for i in range(len(keys)):
    freq_temp2 = math.pow(2.0, keys[i])
    freq = 27.5 * ((freq_temp2) ** (1.0/12.0))
    T = Tbase*Tratios[i]
    aparam = [0.02*T, 1, 0.1] #t1,a1,a2
    wave_temp.append( makeampFnwave(1, freq, T, dT, sr, aparam) )
wave_temp = np.array(list(chain.from_iterable(wave_temp)))

wave0 = wave_temp

wave0max = max(abs(wave0))
print(wave0max)
wave0 = wave0/wave0max

wavwrite(wave0,"is_music.wav")

0.999770575752


In [5]:
# reference
# http://133.88.240.21/~matsunaga/~matsunagarie/index.php?plugin=attach&refer=Publications&openfile=08%E6%9D%BE%E6%B0%B8%E9%98%BF%E9%83%A8.pdf
# fig1